In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql.types import LongType, StringType, BooleanType, StructField, StructType, ArrayType, IntegerType, FloatType, DoubleType
from pyspark.sql.functions import when, col, concat, lit

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/AreasVerdes/warehouse'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
hdfs_path = "hdfs://hdfs-nn:9000/AreasVerdes/bronze/InfQuintais_csv/InfQuintais.csv"

In [3]:
# Carrega o dataset da informação do quintal
customSchema = StructType([
    StructField("ParksID", StringType(), True),        
    StructField("InspectionID", IntegerType(), True),
    StructField("NextToAnotherGarden", BooleanType(), True),
    StructField("TotalFenceLength", FloatType(), True),
    StructField("TotalSidewalkArea", FloatType(), True),
    StructField("TotalSidewalkLength", FloatType(), True),
    StructField("OnSiteService", BooleanType(), True),
    StructField("HydrantW_in15ft", BooleanType(), True),
    StructField("HydrantOnGardenSide", BooleanType(), True),
    StructField("RainHarvesting", BooleanType(), True),
    StructField("RainGallons", FloatType(), True),
    StructField("SolarPanels", BooleanType(), True),
    StructField("CompostSystem",  BooleanType(), True),
    StructField("ComposTumblers", BooleanType(), True),
    StructField("NonFoodPlants", BooleanType(), True),
    StructField("Food", BooleanType(), True),
    StructField("OpenLawnOrCommunalArea", BooleanType(), True),
    StructField("PavedArea", BooleanType(), True),
    StructField("TreesInGarden", BooleanType(), True),
    StructField("FruitTrees", BooleanType(), True),
    StructField("StreetTrees", BooleanType(), True),
    StructField("EmptyTreePits", BooleanType(), True),
    StructField("Murals", BooleanType(), True),
    StructField("BlankShed", BooleanType(), True),
    StructField("ParksSign", BooleanType(), True),
    StructField("Chickens", BooleanType(), True),
    StructField("Pond", BooleanType(), True),
    StructField("FishInPond", BooleanType(), True),
    StructField("Turtles", BooleanType(), True),
    StructField("Aquaponics", BooleanType(), True),
    StructField("FarmersMarket", BooleanType(), True),
    StructField("CSApickup", BooleanType(), True),
    StructField("Composting", BooleanType(), True),
    StructField("GreenHouse", BooleanType(), True),
    StructField("StructureForSeasonExtension", BooleanType(), True)
])

episodes = spark \
            .read\
            .option("delimiter",",")\
            .option("header","true")\
            .schema(customSchema) \
            .csv(hdfs_path)

In [4]:
# mostra a schema e a tabela
episodes.printSchema()
episodes.show()
#episodes.toPandas()

root
 |-- ParksID: string (nullable = true)
 |-- InspectionID: integer (nullable = true)
 |-- NextToAnotherGarden: boolean (nullable = true)
 |-- TotalFenceLength: float (nullable = true)
 |-- TotalSidewalkArea: float (nullable = true)
 |-- TotalSidewalkLength: float (nullable = true)
 |-- OnSiteService: boolean (nullable = true)
 |-- HydrantW_in15ft: boolean (nullable = true)
 |-- HydrantOnGardenSide: boolean (nullable = true)
 |-- RainHarvesting: boolean (nullable = true)
 |-- RainGallons: float (nullable = true)
 |-- SolarPanels: boolean (nullable = true)
 |-- CompostSystem: boolean (nullable = true)
 |-- ComposTumblers: boolean (nullable = true)
 |-- NonFoodPlants: boolean (nullable = true)
 |-- Food: boolean (nullable = true)
 |-- OpenLawnOrCommunalArea: boolean (nullable = true)
 |-- PavedArea: boolean (nullable = true)
 |-- TreesInGarden: boolean (nullable = true)
 |-- FruitTrees: boolean (nullable = true)
 |-- StreetTrees: boolean (nullable = true)
 |-- EmptyTreePits: boolean (

In [5]:
#Escolher as colunas
infquintais = episodes.select("ParksID","InspectionID","TotalFenceLength","TotalSidewalkArea","TotalSidewalkLength","OnSiteService","HydrantW_in15ft","HydrantOnGardenSide","RainHarvesting","RainGallons","SolarPanels","CompostSystem","ComposTumblers","NonFoodPlants","Food","OpenLawnOrCommunalArea","PavedArea","TreesInGarden","FruitTrees","StreetTrees","EmptyTreePits","Murals","BlankShed","ParksSign","Chickens","Pond","Turtles","Aquaponics","FarmersMarket","CSApickup","Composting","Greenhouse","StructureForSeasonExtension")
infquintais.show(truncate=False)

+-----------+------------+----------------+-----------------+-------------------+-------------+---------------+-------------------+--------------+-----------+-----------+-------------+--------------+-------------+-----+----------------------+---------+-------------+----------+-----------+-------------+------+---------+---------+--------+-----+-------+----------+-------------+---------+----------+----------+---------------------------+
|ParksID    |InspectionID|TotalFenceLength|TotalSidewalkArea|TotalSidewalkLength|OnSiteService|HydrantW_in15ft|HydrantOnGardenSide|RainHarvesting|RainGallons|SolarPanels|CompostSystem|ComposTumblers|NonFoodPlants|Food |OpenLawnOrCommunalArea|PavedArea|TreesInGarden|FruitTrees|StreetTrees|EmptyTreePits|Murals|BlankShed|ParksSign|Chickens|Pond |Turtles|Aquaponics|FarmersMarket|CSApickup|Composting|Greenhouse|StructureForSeasonExtension|
+-----------+------------+----------------+-----------------+-------------------+-------------+---------------+-----------

In [6]:
# Substitui os dados a null ou vazios por 0 em caso de float/double/int e False em caso de boolean)
infquintais = infquintais.na.fill(0,["TotalFenceLength"])
infquintais = infquintais.na.fill(0,["TotalSidewalkArea"])
infquintais = infquintais.na.fill(0,["TotalSidewalkLength"])
infquintais = infquintais.na.fill(False,["OnSiteService"])
infquintais = infquintais.na.fill(False,["HydrantW_in15ft"])
infquintais = infquintais.na.fill(False,["HydrantOnGardenSide"])
infquintais = infquintais.na.fill(False,["RainHarvesting"])
infquintais = infquintais.fillna(0,["RainGallons"])
infquintais = infquintais.na.fill(False,["SolarPanels"])
infquintais = infquintais.na.fill(False,["CompostSystem"])
infquintais = infquintais.na.fill(False,["ComposTumblers"])
infquintais = infquintais.na.fill(False,["NonFoodPlants"])
infquintais = infquintais.na.fill(False,["Food"])
infquintais = infquintais.na.fill(False,["OpenLawnOrCommunalArea"])
infquintais = infquintais.na.fill(False,["PavedArea"])
infquintais = infquintais.na.fill(False,["TreesInGarden"])
infquintais = infquintais.na.fill(False,["FruitTrees"])
infquintais = infquintais.na.fill(False,["StreetTrees"])
infquintais = infquintais.na.fill(False,["EmptyTreePits"])
infquintais = infquintais.na.fill(False,["Murals"])
infquintais = infquintais.na.fill(False,["BlankShed"])
infquintais = infquintais.na.fill(False,["ParksSign"])
infquintais = infquintais.na.fill(False,["Chickens"])
infquintais = infquintais.na.fill(False,["Pond"])
infquintais = infquintais.na.fill(False,["Turtles"])
infquintais = infquintais.na.fill(False,["Aquaponics"])
infquintais = infquintais.na.fill(False,["FarmersMarket"])
infquintais = infquintais.na.fill(False,["CSApickup"])
infquintais = infquintais.na.fill(False,["Composting"])
infquintais = infquintais.na.fill(False,["Greenhouse"])
infquintais = infquintais.na.fill(False,["StructureForSeasonExtension"])
infquintais.show()

+-----------+------------+----------------+-----------------+-------------------+-------------+---------------+-------------------+--------------+-----------+-----------+-------------+--------------+-------------+-----+----------------------+---------+-------------+----------+-----------+-------------+------+---------+---------+--------+-----+-------+----------+-------------+---------+----------+----------+---------------------------+
|    ParksID|InspectionID|TotalFenceLength|TotalSidewalkArea|TotalSidewalkLength|OnSiteService|HydrantW_in15ft|HydrantOnGardenSide|RainHarvesting|RainGallons|SolarPanels|CompostSystem|ComposTumblers|NonFoodPlants| Food|OpenLawnOrCommunalArea|PavedArea|TreesInGarden|FruitTrees|StreetTrees|EmptyTreePits|Murals|BlankShed|ParksSign|Chickens| Pond|Turtles|Aquaponics|FarmersMarket|CSApickup|Composting|Greenhouse|StructureForSeasonExtension|
+-----------+------------+----------------+-----------------+-------------------+-------------+---------------+-----------

In [7]:
infquintais = infquintais.withColumn("Borough", when(infquintais.ParksID.startswith('B'), lit ("Brooklyn")) \
  .when (infquintais.ParksID.startswith ('M'), lit ("Manhattan")) \
  .when (infquintais.ParksID.startswith ('Q'), lit ("Queens")) \
  .when (infquintais.ParksID.startswith ('R'), lit ("Staten Island")) \
  .when (infquintais.ParksID.startswith ('X'), lit ("Bronx")))                               
infquintais.show();

+-----------+------------+----------------+-----------------+-------------------+-------------+---------------+-------------------+--------------+-----------+-----------+-------------+--------------+-------------+-----+----------------------+---------+-------------+----------+-----------+-------------+------+---------+---------+--------+-----+-------+----------+-------------+---------+----------+----------+---------------------------+---------+
|    ParksID|InspectionID|TotalFenceLength|TotalSidewalkArea|TotalSidewalkLength|OnSiteService|HydrantW_in15ft|HydrantOnGardenSide|RainHarvesting|RainGallons|SolarPanels|CompostSystem|ComposTumblers|NonFoodPlants| Food|OpenLawnOrCommunalArea|PavedArea|TreesInGarden|FruitTrees|StreetTrees|EmptyTreePits|Murals|BlankShed|ParksSign|Chickens| Pond|Turtles|Aquaponics|FarmersMarket|CSApickup|Composting|Greenhouse|StructureForSeasonExtension|  Borough|
+-----------+------------+----------------+-----------------+-------------------+-------------+-------

In [8]:
infquintais = infquintais.withColumn("Composting",col("Composting").cast(StringType()))
infquintais = infquintais.withColumn("Composting", when((infquintais.CompostSystem == True) & infquintais.ComposTumblers == True, "Both")\
.when ((infquintais.CompostSystem == True) & (infquintais.ComposTumblers == False), "Bins")\
.when ((infquintais.CompostSystem == False) & (infquintais.ComposTumblers == True), "Barrels")\
.when ((infquintais.CompostSystem == False) & (infquintais.ComposTumblers == False), "No Composting System"))
infquintais = infquintais.drop('CompostSystem')
infquintais = infquintais.drop('ComposTumblers')
infquintais.show();

+-----------+------------+----------------+-----------------+-------------------+-------------+---------------+-------------------+--------------+-----------+-----------+-------------+-----+----------------------+---------+-------------+----------+-----------+-------------+------+---------+---------+--------+-----+-------+----------+-------------+---------+--------------------+----------+---------------------------+---------+
|    ParksID|InspectionID|TotalFenceLength|TotalSidewalkArea|TotalSidewalkLength|OnSiteService|HydrantW_in15ft|HydrantOnGardenSide|RainHarvesting|RainGallons|SolarPanels|NonFoodPlants| Food|OpenLawnOrCommunalArea|PavedArea|TreesInGarden|FruitTrees|StreetTrees|EmptyTreePits|Murals|BlankShed|ParksSign|Chickens| Pond|Turtles|Aquaponics|FarmersMarket|CSApickup|          Composting|Greenhouse|StructureForSeasonExtension|  Borough|
+-----------+------------+----------------+-----------------+-------------------+-------------+---------------+-------------------+---------

In [9]:

infquintais = infquintais.withColumn("Plants", when((infquintais.NonFoodPlants == True) & infquintais.Food == True, "Both")\
.when ((infquintais.NonFoodPlants == True) & (infquintais.Food == False), "NonFood")\
.when ((infquintais.NonFoodPlants == False) & (infquintais.Food == True), "Food")\
.when ((infquintais.NonFoodPlants == False) & (infquintais.Food == False), "No Plants"))
infquintais = infquintais.drop('NonFoodPlants')
infquintais = infquintais.drop('Food')
infquintais.show();

+-----------+------------+----------------+-----------------+-------------------+-------------+---------------+-------------------+--------------+-----------+-----------+----------------------+---------+-------------+----------+-----------+-------------+------+---------+---------+--------+-----+-------+----------+-------------+---------+--------------------+----------+---------------------------+---------+-------+
|    ParksID|InspectionID|TotalFenceLength|TotalSidewalkArea|TotalSidewalkLength|OnSiteService|HydrantW_in15ft|HydrantOnGardenSide|RainHarvesting|RainGallons|SolarPanels|OpenLawnOrCommunalArea|PavedArea|TreesInGarden|FruitTrees|StreetTrees|EmptyTreePits|Murals|BlankShed|ParksSign|Chickens| Pond|Turtles|Aquaponics|FarmersMarket|CSApickup|          Composting|Greenhouse|StructureForSeasonExtension|  Borough| Plants|
+-----------+------------+----------------+-----------------+-------------------+-------------+---------------+-------------------+--------------+-----------+------

In [10]:
# As colunas HydrantW_in15ft e RainGallons serão renomeadas para consolidar a mudança de unidades dos valores das duas colunas

infquintais = infquintais.withColumnRenamed("HydrantW_in15ft","HydrantW_in5m")\
                         .withColumnRenamed("RainGallons","RainLitres")
infquintais.show();


+-----------+------------+----------------+-----------------+-------------------+-------------+-------------+-------------------+--------------+----------+-----------+----------------------+---------+-------------+----------+-----------+-------------+------+---------+---------+--------+-----+-------+----------+-------------+---------+--------------------+----------+---------------------------+---------+-------+
|    ParksID|InspectionID|TotalFenceLength|TotalSidewalkArea|TotalSidewalkLength|OnSiteService|HydrantW_in5m|HydrantOnGardenSide|RainHarvesting|RainLitres|SolarPanels|OpenLawnOrCommunalArea|PavedArea|TreesInGarden|FruitTrees|StreetTrees|EmptyTreePits|Murals|BlankShed|ParksSign|Chickens| Pond|Turtles|Aquaponics|FarmersMarket|CSApickup|          Composting|Greenhouse|StructureForSeasonExtension|  Borough| Plants|
+-----------+------------+----------------+-----------------+-------------------+-------------+-------------+-------------------+--------------+----------+-----------+---

In [11]:
# Conversão da coluna "RainLitres" (anteriormente "RainGallons") de galões para litros

infquintais = infquintais.withColumn ("RainLitres", col ("RainLitres")*3.785)
infquintais.show()

# guardar as alteracoes
infquintais_guardar = infquintais.select("ParksID","Borough","InspectionID","TotalFenceLength","TotalSidewalkArea","TotalSidewalkLength","OnSiteService","HydrantW_in5m","HydrantOnGardenSide","RainHarvesting","RainLitres","SolarPanels","Composting","Plants","OpenLawnOrCommunalArea","PavedArea","TreesInGarden","FruitTrees","StreetTrees","EmptyTreePits","Murals","BlankShed","ParksSign","Chickens","Pond","Turtles","Aquaponics","FarmersMarket","CSApickup","Greenhouse","StructureForSeasonExtension")
infquintais_guardar.printSchema()
infquintais_guardar.show()

+-----------+------------+----------------+-----------------+-------------------+-------------+-------------+-------------------+--------------+------------------+-----------+----------------------+---------+-------------+----------+-----------+-------------+------+---------+---------+--------+-----+-------+----------+-------------+---------+--------------------+----------+---------------------------+---------+-------+
|    ParksID|InspectionID|TotalFenceLength|TotalSidewalkArea|TotalSidewalkLength|OnSiteService|HydrantW_in5m|HydrantOnGardenSide|RainHarvesting|        RainLitres|SolarPanels|OpenLawnOrCommunalArea|PavedArea|TreesInGarden|FruitTrees|StreetTrees|EmptyTreePits|Murals|BlankShed|ParksSign|Chickens| Pond|Turtles|Aquaponics|FarmersMarket|CSApickup|          Composting|Greenhouse|StructureForSeasonExtension|  Borough| Plants|
+-----------+------------+----------------+-----------------+-------------------+-------------+-------------+-------------------+--------------+----------

In [12]:
#write df to hive deltalake_table
#.select("customer_name","month", "sales", "year")  reorder columns to match parquet table sequence
infquintais_guardar \
    .select("ParksID","Borough","InspectionID","TotalFenceLength","TotalSidewalkArea","TotalSidewalkLength","OnSiteService","HydrantW_in5m","HydrantOnGardenSide","RainHarvesting","RainLitres","SolarPanels","Composting","Plants","OpenLawnOrCommunalArea","PavedArea","TreesInGarden","FruitTrees","StreetTrees","EmptyTreePits","Murals","BlankShed","ParksSign","Chickens","Pond","Turtles","Aquaponics","FarmersMarket","CSApickup","Greenhouse","StructureForSeasonExtension") \
    .write \
    .mode("overwrite") \
    .format("delta") \
    .save("hdfs://hdfs-nn:9000/AreasVerdes/silver/InfQuintais")

In [14]:
spark.sql(
    """
    SHOW TABLES FROM AreasVerdes
    """
).show()

+-----------+--------------------+-----------+
|  namespace|           tableName|isTemporary|
+-----------+--------------------+-----------+
|areasverdes|       area_sidewalk|      false|
|areasverdes|avg_capacity_capt...|      false|
|areasverdes|   infquintais_table|      false|
|areasverdes|  numero_compostagem|      false|
|areasverdes|      numero_jardins|      false|
|areasverdes|      numero_plantas|      false|
+-----------+--------------------+-----------+



In [42]:
infquintais.createOrReplaceTempView("episodes")

sqlized_df = spark.sql(
    """
    SELECT Plants
    FROM episodes
    """
)

sqlized_df.show()

+-------+
| Plants|
+-------+
|   Both|
|NonFood|
|NonFood|
|   Both|
|   Both|
|   Both|
|   Both|
|   Both|
|NonFood|
|   Both|
|   Both|
|   Both|
|   Both|
|   Both|
|   Both|
|   Both|
|   Both|
|   Both|
|   Both|
|   Both|
+-------+
only showing top 20 rows



In [41]:
spark.stop()